In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
gpu_id = '2'
n_users = 10
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id
dataset_name = 'trentino'
sub_set_name = 'sms'
series_type = 'hourly'

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
def preprocess(data, std=True):
    '''
    Function to do preprocess for input data
    inputs:
        + data: pandas.DataFrame
    outputs:
        + preprocessed_data: pandas.DataFrame
    '''
    if std:
        scaler = StandardScaler()
        temp = scaler.fit_transform(data)
    else:
        scaler = MinMaxScaler()
        temp = scaler.fit_transform(data)
    processed_data = pd.DataFrame(temp, index=data.index, columns = data.columns)
    # processed_data = processed_data.to_numpy()
    return processed_data

In [6]:
file_path = f"./dataset/{dataset_name}/{dataset_name}_{sub_set_name}_{series_type}.csv"
raw_data = pd.read_csv(file_path)
raw_data = raw_data.pivot(index='startTime', columns='gridID', values=sub_set_name)
raw_data = raw_data.fillna(0)
raw_data = preprocess(raw_data, std=False)
raw_data.head()

gridID,38,39,40,154,155,156,157,158,159,160,...,11217,11218,11219,11220,11335,11336,11337,11452,11453,11454
startTime,,,,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00,0.094306,0.080689,0.101714,0.074183,0.074400,0.062342,0.047282,0.096157,0.074058,0.099453,...,0.023615,0.006988,0.003028,0.001136,0.069312,0.007474,0.004286,0.071597,0.023736,0.003482
2013-11-01 01:00:00,0.060986,0.025942,0.081870,0.018331,0.022989,0.026928,0.043612,0.077382,0.081236,0.077416,...,0.008073,0.004614,0.000000,0.000290,0.012389,0.001223,0.001732,0.011477,0.006967,0.002844
2013-11-01 02:00:00,0.053028,0.049953,0.095736,0.045111,0.045652,0.036720,0.046584,0.048535,0.044577,0.011898,...,0.006370,0.001149,0.000000,0.000056,0.004782,0.000258,0.001150,0.003682,0.001755,0.001462
2013-11-01 03:00:00,0.025854,0.029653,0.043593,0.024203,0.031771,0.043355,0.057673,0.039691,0.041080,0.011818,...,0.001424,0.000926,0.000000,0.000000,0.003324,0.000337,0.000415,0.003197,0.002012,0.000710
2013-11-01 04:00:00,0.022510,0.028511,0.029727,0.029609,0.033657,0.033626,0.047155,0.028809,0.044062,0.010069,...,0.001013,0.000607,0.000000,0.000000,0.001461,0.000109,0.000000,0.001616,0.001701,0.000000


In [7]:
# prompt: write pandas to csv

import pandas as pd

# Save the DataFrame to a CSV file
raw_data.rename(columns={'startTime': 'date'}, inplace=True)
raw_data.to_csv(f"./dataset/{dataset_name}/{sub_set_name}.csv", index_label='date')
raw_data.head()

gridID,38,39,40,154,155,156,157,158,159,160,...,11217,11218,11219,11220,11335,11336,11337,11452,11453,11454
startTime,,,,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00,0.094306,0.080689,0.101714,0.074183,0.074400,0.062342,0.047282,0.096157,0.074058,0.099453,...,0.023615,0.006988,0.003028,0.001136,0.069312,0.007474,0.004286,0.071597,0.023736,0.003482
2013-11-01 01:00:00,0.060986,0.025942,0.081870,0.018331,0.022989,0.026928,0.043612,0.077382,0.081236,0.077416,...,0.008073,0.004614,0.000000,0.000290,0.012389,0.001223,0.001732,0.011477,0.006967,0.002844
2013-11-01 02:00:00,0.053028,0.049953,0.095736,0.045111,0.045652,0.036720,0.046584,0.048535,0.044577,0.011898,...,0.006370,0.001149,0.000000,0.000056,0.004782,0.000258,0.001150,0.003682,0.001755,0.001462
2013-11-01 03:00:00,0.025854,0.029653,0.043593,0.024203,0.031771,0.043355,0.057673,0.039691,0.041080,0.011818,...,0.001424,0.000926,0.000000,0.000000,0.003324,0.000337,0.000415,0.003197,0.002012,0.000710
2013-11-01 04:00:00,0.022510,0.028511,0.029727,0.029609,0.033657,0.033626,0.047155,0.028809,0.044062,0.010069,...,0.001013,0.000607,0.000000,0.000000,0.001461,0.000109,0.000000,0.001616,0.001701,0.000000


In [8]:
# Download the dataset
# download_elec_dataset()

# # Clean dataset
# elec_data = clean_elec()

# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Std_normalization = 1
# if Std_normalization:
#     scaler = StandardScaler()
#     temp = scaler.fit_transform(elec_data)
#     norm_means = scaler.mean_
#     norm_std = scaler.scale_
# else:
#     scaler = MinMaxScaler()
#     temp = scaler.fit_transform(elec_data)

# elec_data = pd.DataFrame(elec_data, index=elec_data.index, columns = elec_data.columns)

In [9]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(raw_data.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = gpu_id
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.root_path='./dataset/trentino/'
args.data_path='sms.csv'
args.data_path

'sms.csv'

In [10]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [11]:
test_data[0][0].shape

(96, 6259)

In [12]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, raw_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, raw_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [13]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(625, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(625, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
        

# Setup WANDB

In [14]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 50
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.2
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Milan dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Trentino SMS dataset",
    # Hyperparams
    config={
        "dataset": "Milan Call",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 10000,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [15]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 cost time: 3.1503190994262695
Epoch: 2 cost time: 2.4468789100646973
Epoch: 3 cost time: 2.4487266540527344
Epoch: 4 cost time: 2.632061719894409
Epoch: 5 cost time: 2.62907075881958
Epoch: 1 cost time: 2.5530896186828613
Epoch: 2 cost time: 2.684006452560425
Epoch: 3 cost time: 2.4825761318206787
Epoch: 4 cost time: 2.5154738426208496
Epoch: 5 cost time: 2.4404544830322266


Progress:   0%|          | 0/50 [00:41<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1268 and 1250x64)